In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='2'
#import wget
import sys
sys.path.append('../')
import argparse
import json
import pandas as pd
import random
import numpy as np
import string
import nltk
from functools import partial
import re
from  tqdm import tqdm
import torch
nltk.download('punkt')
from src.dataset_classes import DatasetObject,Features,SquadQuestionGenerationDataset
from src.utils import answerGeneratorDataset,questionGeneratorDataset,buildFact,setuptokenizer,pad_seq,SmartCollator
from dataclasses import dataclass, field
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to /home/nlplab/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
model_base = 't5-base'
tokenizer = setuptokenizer(model_base=model_base,
                           special_tokens=['<section>','</section>'
                                           ,'<generate_questions>',
                                           '<generate_answers>'])

/home/nlplab/anaconda3/envs/development/lib/python3.9/site-packages/transformers/models/t5/tokenization_t5_fast.py:156: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [3]:
dataset = SquadQuestionGenerationDataset(tokenizer,nb_records=1)
dataset.change_data_mode(1)

In [4]:
from src.model_utils import model_init

trained_weights = torch.load('../trained_models/t5_base_model_1/checkpoint-18000/pytorch_model.bin')

generator = model_init(model_base=model_base,vocab_size=len(tokenizer))
generator.load_state_dict(trained_weights)
device = generator.device

In [5]:
from nltk.util import ngrams
import wikipedia
def factgenerator(document,n):
    return list(ngrams(sent_tokenize(document.strip()),n))

In [6]:
sample_too = True
sampling_helper = {} if not sample_too else dict(top_k=30, top_p=0.95,)
max_length=250
length_penalty=2.6
beam_size=4
repetition_penalty=1.56
return_top_beams= beam_size if not sample_too else 10

In [25]:
article = wikipedia.summary('Afrobeat')
n=4
facts = [' '.join(s).replace('\n','').strip() for s in factgenerator(article.replace('.T','. T'),n=n)]

In [29]:
facts

['Afrobeat is a Nigerian music genre that involves the combination of West African musical styles (such as traditional Yoruba music and highlife) and American funk, jazz, and soul influences, with a focus on chanted vocals, complex intersecting rhythms, and percussion. The style was pioneered in the 1960s by Nigerian multi-instrumentalist and bandleader Fela Kuti, who is responsible for popularizing the style both within and outside Nigeria. Distinct from Afrobeat is Afrobeats – a sound originating in West Africa in the 21st century, one that takes in diverse influences and is an eclectic combination of genres such as hip hop, house, jùjú, ndombolo, R&B and soca. The two genres, though often conflated, are not the same.']

In [17]:
from  mosestokenizer.detokenizer import MosesDetokenizer

In [22]:
tokenize = MosesDetokenizer('en')

In [21]:
tokenize('White is an achromatic color , a color without hue .'.split())

'White is an achromatic color, a color without hue.'

In [37]:
a= set()
b= set()

In [44]:
d = {'alice': 18, 'bob': 24, 'carl': 32}
keys = ['carl', 'bobs']
result = list(map(d.get, keys))
print(result)

[32, None]


In [38]:
a.add('Hello')
b.add('World')

In [43]:
', '.join(b.union(a))

'Hello, World'

In [28]:
task_id = 0
target_fact = facts[0]
#'Table 1 Chemical combination rule for working with N2 gas: item[COA]. volume[32m3].  ratio[0.06].  '
#facts[11]

#' item[COA], volume[32m3],  ratio[0.06]  Table 1: Chemical combination rule for working with N2 gas.'
data = DatasetObject(task='<generate_questions> ', question='',
                     context=target_fact, fact=target_fact,
                     answer='',
                     answer_sentence='',
                     task_id="")

batch = dataset.procesTexts(data)

b_input_ids = batch.input_ids.view(1, -1).to(device)
b_input_mask = batch.attention_mask.view(1, -1).to(device)

sample_too = True
sampling_helper = {} if not sample_too else dict(top_k=25, top_p=0.95,)
return_top_beams = beam_size if not sample_too else 25
# seed_everything(2982)
generator.eval()
with torch.no_grad():
    sample_outputs = generator.generate(input_ids=b_input_ids,  **sampling_helper,
                                        attention_mask=b_input_mask,
                                        num_beams=beam_size,
                                        repetition_penalty=repetition_penalty,
                                        length_penalty=length_penalty,
                                        early_stopping=False,
                                        use_cache=True,
                                        max_length=max_length,
                                        no_repeat_ngram_size=2,
                                        num_return_sequences=return_top_beams,
                                        do_sample=sample_too,
                                        eos_token_id=dataset.tokenizer.eos_token_id,)
oop = [dataset.tokenizer.decode(sample_outputs[idx],
                                skip_special_tokens=True,
                                clean_up_tokenization_spaces=True) for idx in range(return_top_beams)]

print(f'Article Section: {data.context}')
print('Questions Generated')
oop = set(oop)
for q in oop:
    print(q)

IndexError: list index out of range

In [21]:
for q in oop:
    print(q)

What is the name of the pancake that is made with buttermilk in place of eggs?
What is the name of the type of pancake made with buttermilk in place of eggs?
What is the pancake with buttermilk in place of or in addition to milk known as in Scotland and the US?
What is the name of the pancake that is made with buttermilk in place of eggs when it is used in addition to milk?
What is the name of the pancake that is used in place of buttermilk in addition to milk?
What is the name of the pancake that is used in place of buttermilk in addition to milk?
What is the name of the type of pancake that is made with buttermilk in place of eggs when it is used in addition to milk?
What is the name of the pancake that is used in place of buttermilk when it is not used?
What type of pancake can be used in place of buttermilk to make it tart?
What is a type of buckwheat pancake that can be used in place of buttermilk or in addition to milk?
What is the name of the pancake that is used when buttermilk